In [1]:
# %load C60_jupyter_imports.py
%load_ext autoreload
%autoreload 2

import xarray as xr
import numpy as np
from dask.distributed import Client
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy import stats
import xesmf as xe
from scipy.stats import linregress
import os
import dask
import requests
# Lets import our functions. There are no tests here. Stored separately so as to not make a mess.
# Make sure in right dir first

os.chdir('/g/data/xv83/np1383/src_CAFE60_eqpac_analysis_code/')

In [2]:
bgc=xr.open_zarr('/g/data/xv83/dcfp/CAFE60v1/ocean_bgc_month.zarr.zip')
bgc=bgc.rename({'xt_ocean':'lon','yt_ocean':'lat'})
bgc['lon']=bgc['lon']+360
eqpac=bgc.sel(lon=slice(120,290),lat=slice(-40,40))
#eqpac.time
eqpac

,Array,Chunk
Bytes,306.42 GiB,26.15 MiB
Shape,"(720, 96, 50, 140, 170)","(1, 96, 3, 140, 170)"
Count,24481 Tasks,12240 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,306.42 GiB,26.15 MiB
Shape,"(720, 96, 50, 140, 170)","(1, 96, 3, 140, 170)"
Count,24481 Tasks,12240 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [3]:
# Set up the remote dask cluster. Can either use this version or a similar version above if building a LocalCluster.
from dask.distributed import Client,Scheduler
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(cores=4,processes=1,memory="47GB")
client = Client(cluster)
cluster.scale(cores=32)
#cluster.adapt(minimum=2, maximum=16)
client

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.07/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46625 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/46625/status,
Dashboard: /proxy/46625/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.0.128.146:34699,Workers: 0
Dashboard: /proxy/46625/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [4]:
cluster

In [5]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/46625/status,
Dashboard: /proxy/46625/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.0.128.146:34699,Workers: 0
Dashboard: /proxy/46625/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [6]:
cafe=['datasets/npp_satellite/cafe_mod_nc/*','datasets/npp_satellite/cafe_sw_nc/*']
#title=['vgpm','cbpm','eppley',
title=['cafe'] #
npps=[[]]#,[],[],[]]

#with dask.config.set(**{'array.slicing.split_large_chunks': True}):

for ix,npp_models in enumerate([cafe]): #vgpm,cbpm,eppley,
    model_holder=[]
    for i, model_fp in enumerate(npp_models):
        #if i>=6:
        #    break
        model=xr.open_mfdataset(model_fp,concat_dim='time',combine='nested')
        name=model_fp.split('/')[-2][:-3]
        model=model.rename({'npp':name})
        print(name+' '+str(model.nbytes/1e9)+' GB')

        # drop duplicate times https://stackoverflow.com/a/51077784/
        _, index = np.unique(model['time'], return_index=True)
        model=model.isel(time=index)

        model_holder.append(model)
    npps[ix]=xr.merge(model_holder)


    _, index = np.unique(npps[ix]['lon'], return_index=True)
    npps[ix]=npps[ix].isel(lon=index)
    mean = npps[ix].to_array(dim='new').mean('new')
    npp=npps[ix].assign(avg_npp=mean).avg_npp

cafe_mod 1.690118104 GB
cafe_sw 1.034051424 GB


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.07/lib/python3.9/site-packages/xarray/core/indexing.py:1226: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.07/lib/python3.9/site-packages/xarray/core/indexing.py:1226: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ... 

In [7]:
npp

<xarray.DataArray 'avg_npp' (time: 1084, lat: 480, lon: 1019)>
dask.array<mean_agg-aggregate, shape=(1084, 480, 1019), dtype=float32, chunksize=(1, 480, 1019), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1997-09-14 1997-09-22 ... 2021-04-23
  * lat      (lat) float64 39.95 39.79 39.62 39.45 ... -39.62 -39.79 -39.95
  * lon      (lon) float64 120.1 120.3 120.5 120.6 ... 289.4 289.6 289.7 289.9

In [8]:
npp.to_netcdf('../processed_data/npp_rg/avg_npp_'+title[ix]+'.nc',engine='h5netcdf')

In [7]:
vgpm=['datasets/npp_satellite/vgpm_mod_nc/*',
        'datasets/npp_satellite/vgbm_sw_nc/*',
        'datasets/npp_satellite/vgpm_viirs_nc/*']

cbpm=['datasets/npp_satellite/cbpm_mod_nc/*',
        'datasets/npp_satellite/cbpm_sw_nc/*',
        'datasets/npp_satellite/cbpm_viirs_nc/*']

eppley=['datasets/npp_satellite/eppley_mod_nc/*',
        'datasets/npp_satellite/eppley_sw_nc/*',  
        'datasets/npp_satellite/eppley_viirs_nc/*']
cafe=['datasets/npp_satellite/cafe_mod_nc/*','datasets/npp_satellite/cafe_sw_nc/*']
#title=['vgpm','cbpm','eppley',
title=['cafe'] #
npps=[[]]#,[],[],[]]

#with dask.config.set(**{'array.slicing.split_large_chunks': True}):

for ix,npp_models in enumerate([cafe]): #vgpm,cbpm,eppley,
    model_holder=[]
    for i, model_fp in enumerate(npp_models):
        #if i>=6:
        #    break
        model=xr.open_mfdataset(model_fp,concat_dim='time',combine='nested')
        name=model_fp.split('/')[-2][:-3]
        model=model.rename({'npp':name})
        print(name+' '+str(model.nbytes/1e9)+' GB')

        # drop duplicate times https://stackoverflow.com/a/51077784/
        _, index = np.unique(model['time'], return_index=True)
        model=model.isel(time=index)

        model_holder.append(model)
    npps[ix]=xr.merge(model_holder)


    _, index = np.unique(npps[ix]['lon'], return_index=True)
    npps[ix]=npps[ix].isel(lon=index)
    mean = npps[ix].to_array(dim='new').mean('new')
    npp=npps[ix].assign(avg_npp=mean).avg_npp

    #USE XESMF to regrid
    #Used Bilinear previously however we need to use conservative method as below
    # regridder = xe.Regridder(npp, eqpac, 'bilinear',reuse_weights=True)
    # npp1d=regridder(npp)

    # npp1=npp1d.resample(time='M').mean(dim='time') 
    # npp1['time']=npp1.time.astype('datetime64[M]')#_rg #First day of month

    # npp1.to_netcdf('datasets/npp_satellite/avg_npp_rg_'+title[ix]+'.nc')

    #Need to add boundary coordinates to perform conservative regridding.
    mod=npp.where(npp>0).to_dataset()
    mod=mod.sortby(['lat'], ascending=True) #This is backwards and was doing everything upside down. 
    lat_rad=eqpac.lat.diff(dim='lat').mean().values/2
    lon_rad=eqpac.lon.diff(dim='lon').mean().values/2
    landlats=np.linspace(eqpac.lat.min().values-lat_rad,eqpac.lat.max().values+lat_rad,len(eqpac.lat)+1)
    landlons=np.linspace(eqpac.lon.min().values-lon_rad,eqpac.lon.max().values+lon_rad,len(eqpac.lon)+1)

    lat_rad=mod.lat.diff(dim='lat').mean().values/2
    lon_rad=mod.lon.diff(dim='lon').mean().values/2
    lats=np.linspace(mod.lat.min().values-lat_rad,mod.lat.max().values+lat_rad,len(mod.lat)+1)
    lons=np.linspace(mod.lon.min().values-lon_rad,mod.lon.max().values+lon_rad,len(mod.lon)+1)

    x=np.meshgrid(lats,lons)
    #mod=mod.expand_dims({'latb':lats})#,'lonb':lons})
    #mod.coords('lon_b')=lats#=(['lonb','latb'],x[0])
    mod.coords['lon_b']=lons
    mod.coords['lat_b']=lats
    mod['lat_b']=lats#(['lonb','latb'],x[1])
    mod['lon_b']=lons#(['lonb','latb'],x[0])
    mod['lat']=mod.lat

    eqpac.coords['lon_b']=landlons
    eqpac.coords['lat_b']=landlats
    eqpac['lat_b']=landlats#(['lonb','latb'],x[1])
    eqpac['lon_b']=landlons##(['lonb','latb'],x[0])


    # Regridding
    print('regridding')
    regridder = xe.Regridder(mod, eqpac, 'conservative',reuse_weights=False)
    npp1d=regridder(mod.chunk(dict(lat=-1)).chunk(dict(lon=-1)))

    npp1=npp1d.resample(time='M').mean(dim='time') 
    npp1['time']=npp1.time.astype('datetime64[M]')#_rg #First day of month
    print('saving')

    npp1.to_netcdf('../processed_data/npp_rg/avg_npp_rg_'+title[ix]+'.nc',engine='h5netcdf')

cafe_mod 1.690118104 GB
cafe_sw 1.034051424 GB


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.07/lib/python3.9/site-packages/xarray/core/indexing.py:1226: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.07/lib/python3.9/site-packages/xarray/core/indexing.py:1226: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ... 

regridding


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.07/lib/python3.9/site-packages/dask/array/gufunc.py:458: PerformanceWarning: Increasing number of chunks by factor of 840
  tmp = blockwise(


saving


In [ ]:
cluster.scale(2)

In [ ]:
#a=xr.open_dataset('../processed_data/npp_rg/avg_npp_rg_cafe.nc')

In [ ]:
#a.avg_npp.mean(dim='time').plot()

In [ ]:
#SST Regrid


In [ ]:
sst = xr.open_dataset('../external_data/sst/sst.mnmean.nc')
sst= sst.assign_coords(lon=(sst.lon % 360)).roll(lon=(sst.dims['lon']),roll_coords=False).sortby('lon')		#EPIC 1 line fix for the dateline problem.
sst=sst.sel(lon=slice(120,290),lat=slice(40,-40)).sst

regridder = xe.Regridder(sst, eqpac, 'bilinear',reuse_weights=False)
sst_rg=regridder(sst)

sst_rg=sst_rg.resample(time='M').mean(dim='time') 
sst_rg['time']=sst_rg.time.astype('datetime64[M]')#_rg #First day of month
sst_rg.to_netcdf('../processed_data/rey_eqpac_sst_rg.nc',engine='h5netcdf')

In [ ]:
mod_tpca=xr.open_mfdataset('../external_data/chl/TPCA/modis/*nc')

In [ ]:
sw_tpca=xr.open_mfdataset('../external_data/chl/TPCA/seawifs/*nc')

In [ ]:
mod_tpca=mod_tpca.resample(time='M').mean(dim='time') 
mod_tpca['time']=mod_tpca.time.astype('datetime64[M]')#_rg #First day of month
sw_tpca=sw_tpca.resample(time='M').mean(dim='time') 
sw_tpca['time']=sw_tpca.time.astype('datetime64[M]')#_rg #First day of month

In [ ]:
mod_tpca=mod_tpca.rename({'chl_tpca':'mod_tpca'})
sw_tpca=sw_tpca.rename({'chl_tpca':'sw_tpca'})

In [ ]:
tpca=xr.merge([mod_tpca.mod_tpca,sw_tpca.sw_tpca]) #,compat='override'

mean_tpca = tpca.to_array(dim='new').mean('new')
#tpca=.assign(avg_npp=mean).avg_npp
tpca['mean_tpca']=mean_tpca

In [ ]:
regridder = xe.Regridder(tpca, eqpac, 'bilinear',reuse_weights=False)
tpca_rg=regridder(tpca.mean_tpca)

In [ ]:
regridder = xe.Regridder(tpca, eqpac, 'bilinear',reuse_weights=False)
tpca_mod_rg=regridder(tpca.mod_tpca)
tpca_sw_rg=regridder(tpca.sw_tpca)

In [ ]:
tpca_rg.to_netcdf('../processed_data/obs/TPCA_month_regrid.nc')
tpca_mod_rg.to_netcdf('../processed_data/obs/TPCA_modis_month_regrid.nc')
tpca_sw_rg.to_netcdf('../processed_data/obs/TPCA_sw_month_regrid.nc')

In [ ]:
tpca_rg.chl_tpca.mean(dim='time').plot()